install clang version 9

In [ ]:
!wget https://apt.llvm.org/llvm.sh
!chmod +x llvm.sh
!./llvm.sh 9

verify they have been installed

In [ ]:
!/usr/bin/clang-9 --version
!/usr/bin/clang++-9 --version

clone the arena size calculator

In [ ]:
!git clone --recurse-submodules https://github.com/edgeimpulse/tflite-find-arena-size
import os
os.chdir("tflite-find-arena-size")

point the makefile to version 9 of clang (ideally this would be done with update-alternatives or `llvm-defaults` but this is an easy hack)

In [ ]:
newmake = """NAME = find-arena-size

CC = /usr/bin/clang-9
CXX = /usr/bin/clang++-9
CFLAGS ?= -Wall

MACROS += -DTF_LITE_DISABLE_X86_NEON
CXXFLAGS += -std=c++11
CFLAGS += -I.
CFLAGS += -Isource
CFLAGS += -Iedge-impulse-sdk/
CFLAGS += -Iedge-impulse-sdk/tensorflow
CFLAGS += -Iedge-impulse-sdk/third_party
CFLAGS += -Iedge-impulse-sdk/third_party/flatbuffers
CFLAGS += -Iedge-impulse-sdk/third_party/flatbuffers/include
CFLAGS += -Iedge-impulse-sdk/third_party/flatbuffers/include/flatbuffers
CFLAGS += -Iedge-impulse-sdk/third_party/gemmlowp/
CFLAGS += -Iedge-impulse-sdk/third_party/gemmlowp/fixedpoint
CFLAGS += -Iedge-impulse-sdk/third_party/gemmlowp/internal
CFLAGS += -Iedge-impulse-sdk/third_party/ruy
CFLAGS += -Imodel-parameters
CFLAGS += -Iedge-impulse-sdk/porting

all: build

.PHONY: build clean

build:
	mkdir -p build
	$(CC) -c $(MACROS) $(CFLAGS) $(LFLAGS) edge-impulse-sdk/tensorflow/lite/c/common.c -o build/common.o
	$(CXX) $(MACROS) $(CXXFLAGS) $(CFLAGS) $(LFLAGS) source/*.cpp edge-impulse-sdk/tensorflow/lite/kernels/*.cc edge-impulse-sdk/tensorflow/lite/kernels/internal/*.cc edge-impulse-sdk/tensorflow/lite/micro/kernels/*.cc edge-impulse-sdk/tensorflow/lite/micro/*.cc edge-impulse-sdk/tensorflow/lite/micro/memory_planner/*.cc edge-impulse-sdk/tensorflow/lite/core/api/*.cc ./edge-impulse-sdk/dsp/memory.cpp edge-impulse-sdk/porting/posix/*.c* build/common.o -o build/$(NAME)

clean:
	rm $(NAME)"""

from pathlib import Path
Path("Makefile").write_text(newmake)

build the calculator

In [ ]:
!make

Now drag your `.tflite` file (e.g., `person_detection.tflite`) into the file browser window on the left. You can then run the following cell to output the arena size:

In [ ]:
!build/find-arena-size /content/person_detection.tflite